# TO DO

1) extract more relevant data points
2) parse dict data from columns (before conversion to DF? after conversion to DF? identify whats most efficient)
3) identify latest offer published (max date)
4) create an airflow project that runs on demand: runs the script, adds records to the DB, uploads to PostgreSQL database

# Connection a l'API France Travail

Process:
1) Request a token for the API you want to work with
2) Retrieve the token
3) Use it to communicate with your target API

# Imports

In [1]:
# Import required libraries
import os
import requests
from requests.exceptions import HTTPError
from datetime import datetime, timedelta
import pandas as pd

# Identifiers

In [2]:
# reading env variables
client_id = os.getenv("FT_API_CLIENT_ID")
client_secret = os.getenv("FT_API_CLIENT_SECRET")

In [3]:
client_id

'PAR_ftairflow_b62948d2c3e79045b6caac2ffb330e6b22598b7481704cc4082d8338e8b3028b'

In [4]:
client_secret

'db4c1c00af8d829b570f070f71a41f245d0c1c6db746442b3d7e10da07cd7075'

# Getting an API access token

In [5]:
# URL to get an access token
# specify the realm at the end of the endpoint
url = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=%2Fpartenaire"

# POST request parameters
data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "api_offresdemploiv2 o2dsoffre",
}

# Headers
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "francetravail.io"
}

# API request
response = requests.post(
    url, 
    data=data, 
    headers=headers
)

# if positive response, store token
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Token successfully granted:", access_token)
else:
    print("Error when retrieving the API token:", response.text)

Token successfully granted: IOLUkS3XEC6uZL3Qm-_HGY4k6lc


In [6]:
# casting dates to isoformat
min_creation_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%dT%H:%M:%SZ") # 30 days ago
max_creation_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ") # today

# API endpoint
offres_emplois_url = "https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"

# headers
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}

# parameters
# looking for
    # data related
    # in two specific departments (Paris and Hauts de Seine)
    # permanent contract (CDI)
    # created in the past 90 days
    # senior experience (3 is the max value = over 3 years of XP)
params = {
    "motsCles": "data",
    "departement": "75,92",
    "typeContrat": "CDI",
    "minCreationDate": str(min_creation_date),
    "maxCreationDate": str(max_creation_date),
    "experience": "3",
}

try:
    response = requests.get(
        offres_emplois_url, 
        params=params, 
        headers=headers
    )
    response.raise_for_status()
except HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")

offres = response.json()

In [7]:
# creates a dataframe with the keys below (considered relevant)

selected_keys = [
    "intitule", 
    "description", 
    "dateCreation",
    "appellationlibelle",
    "secteurActiviteLibelle",
    "typeContratLibelle",
    "salaire"
]

filtered_list = [
    {key: offre[key] for key in selected_keys if key in offre}
    for offre in offres["resultats"]
]

offres_df = pd.DataFrame(filtered_list)

In [8]:
# some data processing
offres_df["dateCreation"] = pd.to_datetime(offres_df["dateCreation"]).dt.date

# get the max creation date for offers
max_date = offres_df["dateCreation"].max()

In [9]:
offres_df

,intitule,description,dateCreation,appellationlibelle,secteurActiviteLibelle,typeContratLibelle,salaire
0,CHEF DE PROJET DATA ET EMPLOI & FORMATION H/F ...,Vous aimez donner du sens aux données et pilot...,2025-09-24,Chargé / Chargée de projet Data,Activités des agences de travail temporaire,CDI,{'libelle': 'Mensuel de 50000.0 Euros à 55000....
1,Data Analyst (H/F),Wewyse est un cabinet de conseil spécialisé en...,2025-09-22,Data analyst,"Ingénierie, études techniques",CDI,{'libelle': 'Annuel de 45000.0 Euros à 50000.0...
2,Ingénieur Commissioning - Data Center F/H (H/F),Dans un contexte de croissance et de structura...,2025-09-22,Ingénieur(e) d'essais en études et développeme...,"Ingénierie, études techniques",CDI,{'commentaire': 'Selon profil'}
3,Chef de projet BI - Power BI Data Fabric.et SA...,Rattaché(e) au directeur des systèmes d'inform...,2025-09-19,Chargé / Chargée de projet Data,Commerce de gros (commerce interentreprises) d...,CDI,{'commentaire': 'salaire sur 13 mois plus vari...
4,Lead Data Scientist AI (H/F),Nous recrutons un(e) Lead Data Scientist AI po...,2025-09-16,Ingénieur / Ingénieure data scientist,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...
5,DATA SCIENTIST (H/F),Audiens est le partenaire de confiance des pro...,2025-09-12,Data scientist,Activités des agences de placement de main-d'œ...,CDI,{'libelle': 'Annuel de 55000.0 Euros à 60000.0...
6,Data Engineer Kafka / BigQuery (H/F),Nouvelle Opportunité - Data Engineer Kafka / B...,2025-09-11,Data engineer,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...
7,Chef de projet DATA (H/F),Vous souhaitez nous accompagner dans notre cro...,2025-09-11,Chef / Cheffe de projet informatique,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 35000.0 Euros à 70000.0...
8,Senior Data Scientist (H/F),Descriptif du poste\n\nAu sein de l'équipe Dat...,2025-09-10,Data scientist,Édition de logiciels applicatifs,CDI,{'libelle': 'Annuel de 45000.0 Euros à 55000.0...
9,Data Scientist - Senior (H/F),Nous recrutons un(e) Data Scientist Senior pou...,2025-09-09,Data scientist,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...


In [16]:
# filters on row 1, column salaire value and shows the data
offres_df.iloc[3, 6]

{'commentaire': 'salaire sur 13 mois plus variable',
 'complement1': 'Primes',
 'complement2': 'Intéressement / participation',
 'listeComplements': [{'code': '1', 'libelle': 'Primes'},
  {'code': '10', 'libelle': 'Intéressement / participation'},
  {'code': '4', 'libelle': 'Ordinateur portable'},
  {'code': '8', 'libelle': 'Restauration'},
  {'code': '3', 'libelle': 'Téléphone mobile'}]}

In [8]:
# filters on row 1, column salaire value and shows the data




offres_df.iloc[1, "salaire"]

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [7]:
# parses the data in salaire column that is a json/dict structure


,intitule,description,dateCreation,appellationlibelle,secteurActiviteLibelle,typeContratLibelle,salaire
0,CHEF DE PROJET DATA ET EMPLOI & FORMATION H/F ...,Vous aimez donner du sens aux données et pilot...,2025-09-24,Chargé / Chargée de projet Data,Activités des agences de travail temporaire,CDI,{'libelle': 'Mensuel de 50000.0 Euros à 55000....
1,Data Analyst (H/F),Wewyse est un cabinet de conseil spécialisé en...,2025-09-22,Data analyst,"Ingénierie, études techniques",CDI,{'libelle': 'Annuel de 45000.0 Euros à 50000.0...
2,Ingénieur Commissioning - Data Center F/H (H/F),Dans un contexte de croissance et de structura...,2025-09-22,Ingénieur(e) d'essais en études et développeme...,"Ingénierie, études techniques",CDI,{'commentaire': 'Selon profil'}
3,Chef de projet BI - Power BI Data Fabric.et SA...,Rattaché(e) au directeur des systèmes d'inform...,2025-09-19,Chargé / Chargée de projet Data,Commerce de gros (commerce interentreprises) d...,CDI,{'commentaire': 'salaire sur 13 mois plus vari...
4,Lead Data Scientist AI (H/F),Nous recrutons un(e) Lead Data Scientist AI po...,2025-09-16,Ingénieur / Ingénieure data scientist,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...
5,DATA SCIENTIST (H/F),Audiens est le partenaire de confiance des pro...,2025-09-12,Data scientist,Activités des agences de placement de main-d'œ...,CDI,{'libelle': 'Annuel de 55000.0 Euros à 60000.0...
6,Data Engineer Kafka / BigQuery (H/F),Nouvelle Opportunité - Data Engineer Kafka / B...,2025-09-11,Data engineer,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...
7,Chef de projet DATA (H/F),Vous souhaitez nous accompagner dans notre cro...,2025-09-11,Chef / Cheffe de projet informatique,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 35000.0 Euros à 70000.0...
8,Senior Data Scientist (H/F),Descriptif du poste\n\nAu sein de l'équipe Dat...,2025-09-10,Data scientist,Édition de logiciels applicatifs,CDI,{'libelle': 'Annuel de 45000.0 Euros à 55000.0...
9,Data Scientist - Senior (H/F),Nous recrutons un(e) Data Scientist Senior pou...,2025-09-09,Data scientist,Conseil en systèmes et logiciels informatiques,CDI,{'libelle': 'Annuel de 45000.0 Euros à 65000.0...


# WORK IN PROGRESS

In [ ]:
# let's check the different keys in a job offer
# we will analyze a single offer and retrieve all of its keys
keys = list(offres["resultats"][0].keys())

for key in keys:
    print(key)